<table>
<tr>
    <td width=10%>
        <img src="images/eisti_logo.png">
    </td>
    <td>
        <center>
            <h1>Deep Learning et Applications</h1>
        </center>
    </td>
    <td width=15%>
        Yann Vernaz  et Paul Gay
    </td>
</tr>
</table>

<br/>
<div id="top"></div>
<center>
    <a style="font-size: 20pt; font-weight: bold">Structure from motion with objects</a>
</center></a>
<br/>

Ce TP vous propose d'implémenter une chaine de traitement complète de structure from motion. Nous allons voir comment des méthodes de Deep Learning peuvent nous aider à obtenir une représentation "sémantique" de la scène en détectant les objets qui la composent. Nous appliquerons ensuite les principes de géométrie vue en classe pour obtenir une représentation 3D de notre séquence d'images. Ceci nous permettra d'avoir un premier exemple de vision par ordinateur.

<br/>


<br/>
&nbsp;&nbsp;&nbsp; 1) <a href="#2D3D"> De la 3D vers la 2D : Comprendre les matrices de projections </a><br/>

Le but de cette partie est de se familiariser avec les formes géomètriques des coniques et des quadriques qui seront utilisées pour représenter des objets. L'intérêt de cette représentation est que les relations de projection entre la 3D et la 2D peuvent être modélisée relativement facilement par des matrices de caméra orthographiques.

&nbsp;&nbsp;&nbsp; 2) <a href="#yolo"> Mesures : détecter les objets sur les images </a><br/>
Il s'agit ensuite de détecter les objets dans des images. Nous allons pour cela utiliser un réseau de neurones convolutionnel célèbre pour sa rapidité : YOLO. 

&nbsp;&nbsp;&nbsp; 3) <a href="#tracking"> Tracking : Associer les détections entre les images </a><br/>
Comme pour les correspondances entre les points, nous allons associer les apparitions du même objet entre différentes images.

&nbsp;&nbsp;&nbsp; 4) <a href="#sfmo"> Retrouver les ellipsoides 3D</a><br/>
A présent que nous avons obtenus nos observations, il s'agit de mettre en forme le système d'équations et de le résoudre. Les étapes sont décrites dans le cours et sur ce notebook. 

##  <a id="3D2D"> 1) Comprendre les matrices de projections </a>

Prenez le temps de vous familiariser avec les fonctions en générant et visualisant des données synthétiques.

Note que vous pouvez avoir un meilleur rendu en exécutant le script depuis un terminal. En effet, les fontions de visualisation 3D sont mal gếrées par le notebook.

In [ ]:
from tp_geometry import gen_synth
from tp_geometry import visualize
import numpy as np

# Génére des données synthétiques avec 5 frames et 3 ellipsoides et une caméra orthographique
n_o = 3 # number of objects
n_f = 5 # number of frames
(Ps, Qs, Cs) = gen_synth.get_conics_quadrics_cameras(typ='orth', n_f=n_f, n_o = n_o)
colors = np.random.rand(n_o, 3) # setting the colors of the ellipsoids
visualize.plot_ellipsoids(Qs, colors = colors, keep_alive=True)
visualize.plot_ellipses(Cs, colors = colors)


Vérfier la relation entre les caméras, les coniques et les quadriques, c'est à dire l'équation : 
$$C = PQP^T $$

Les matrices sont stockées par blocs dans les variables `Ps`, `Cs` et `Qs`.
```
Ps[:2,:] # deux premières lignes de la matrice de rotation de la caméra pour la première image
Cs[:3,:3] # conique du premier objet dans la première image
Qs[:4,:] # quadrique du premier objet
```

Note: afin d'effectuer l'opération, vous devez récupérer les deux lignes et ajouter des 1 et 0 comme indiquer dans le cours pour que les dimensions correspondent.

In [ ]:
print("Le premier conique de la première image")
print(C[:3,:3])
print("la reprojection du quadrique")
# votre code ici
reprojection = 
print(reprojection/(-reprojection[2,2]))

##  <a id="yolo"> 2) Mesures : détecter les objets sur les images</a>

À l'heure actuelle le modèle Yolo est l'un des réseaux de neurones les plus utilisés pour détecter des objets dans les images. En particulier, c'est l'un des plus rapides. Il sera détaillé dans le cours sur la détection d'objets. 

Téléchargez les poids du réseau de neurones yolov3. Ils ont déjà été optimisés sur des bases standards de vision par ordinateur et peuvent être utilisés directement pour l'inférence :
```
wget https://pjreddie.com/media/files/yolov3.weights
```

Nous allons à présent le tester rapidement sur une image. Tout d'abord nous devons écrire les fonctions qui appliqueront l'inférence, et afficheront les résultats sur l'image sous forme de boites englobantes. 


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt, matplotlib

# function to get the output layer names 
# in the architecture
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

def get_detection(image, net):
    Width = image.shape[1]
    Height = image.shape[0]
    scale = 0.00392
   # create input blob 
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    # set input blob for the network
    net.setInput(blob)
    # run inference through the network
    # and gather predictions from output layers
    outs = net.forward(get_output_layers(net))
    # initialization
    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4

    # for each detetion from each output layer 
    # get the confidence, class id, bounding box params
    # and ignore weak detections (confidence < 0.5)
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
    # apply non maxima suppression to remove duplicate detections 
    # i.e. the same object has been detected multiple times
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    boxes = [boxes[i[0]] for i in indices]
    confidences = [confidences[i[0]] for i in indices]
    class_ids = [class_ids[i[0]] for i in indices]
    return boxes, confidences, class_ids


# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, label, x, y, x_plus_w, y_plus_h, color, width=5):
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, width)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
def draw_detections(image, class_ids, classes, boxes, COLORS):
    # generate different colors for different classes 
    # go through the detections remaining
    # after nms and draw bounding box
    for (i,box) in enumerate(boxes):
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        class_id = class_ids[i]
        label = classes[class_id]
        color = COLORS[class_id]
        draw_bounding_box(image, label, round(x), round(y), round(x+w), round(y+h), color)  
    return image

Appliquer le code suivant sur l'image de votre choix pour visualiser les détections, en ayant modifié les chemins en fonction de votre configuration

In [ ]:
image_path = "/home/paul/libs/darknet/data/dog.jpg" # input image
darknet_dir = "/home/paul/libs/darknet/" 
config_yolo = darknet_dir + "/cfg/yolov3.cfg" # the description of the network, layer by layer.
class_names = darknet_dir + "/data/coco.names" # the list of classes this network can detect
weights = "/home/paul/data/eisti/Image_processing_geometry/yolov3.weights"

# read class names from text file
classes = None
with open(class_names, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

image = cv2.imread(image_path)
# read pre-trained model and config file
net = cv2.dnn.readNet(weights, config_yolo)
boxes, confidences, class_ids = get_detection(image, net)
COLORS = np.random.uniform(0, 255, size=(len(classes), 3)) # randomly draw a color for each detection     
image = draw_detections(image, class_ids, classes, boxes, COLORS)
matplotlib.rcParams['figure.figsize'] = [15, 15]
_ = plt.imshow(image)

Le script `yolo_on_whole_seq.py` extraiera les détections pour tout les fichiers png présents dans le répertoire indiqué dans son code. 
Les résultats seront sauvegardés au format json dans le fichier `yolo_detections.json`.

Modifiez le script en fonction de votre configuration et utlisez-le pour détecter les objets dans les images.

Regardons les résultats pour quelques-unes des images: 

In [ ]:
import json

json_file = '/home/paul/data/eisti/Image_processing_geometry/yolo_detections.json'
results = json.load(open(json_file)) 

bottle_class = 39 # in this exercice, we only consider this class
image_paths = list(results.keys())
fig=plt.figure(figsize=(15, 10))
rows, cols = 3, 3
for i, image_path in enumerate(image_paths[:9]):
    (bbox, conf, class_names) = results[image_path]
    bottles = [  box  for  (i, box) in enumerate(bbox)  if class_names[i] == 'bottle' ]
    class_ids = [bottle_class for _ in bottles]
    image = cv2.imread(image_path)
    image = draw_detections(image, class_ids, classes,  bottles, COLORS)
    fig.add_subplot(rows, cols, i+1)
    plt.axis('off')
    plt.imshow(image)
    

##  <a id="tracking"> 3) Tracking : Associer les détections entre les images</a>

Dans l'étape précedente, nous avons détecté les bouteilles indépendamment dans chaque image. Mais afin de raisonner géométriquement sur les différentes vues de chaque objet, il nous faut associer les détections de chaque bouteille dans les différentes images dans une même liste. 


Le script `tracking.py` lit le ficher précédemment créé, associe les détections entre elles et enregistre le résultat dans un fichier `tracking.json`.

EXERCICE: Vous devez le compléter en écrivant la fonction qui calcule une ressemblance entre deux détections.

Une fois que vous l'avez fait, vous pouvez visualiser les résultats sur avec la cellule suivante. Normalement, chaque objet doit être associé à la même couleur quelle que soit les images.

Note: pour visualiser toutes les images, vous pouvez utiliser le script `visualise_tracking.py` qui générera les ~800 images de la séquence dans le répertoire courant.

In [ ]:
tracking_results = json.load(open('/home/paul/Documents/cours/eisti/2020_21/04-Image-Processing-Advanced/labs/tracking.json'))

image_paths = list(tracking_results.keys())
# randomly draw a color for each track (assuming 10 tracks)
COLORS_TRACKING =  np.random.uniform(0, 255, size=(10, 3))
track_names = ['track_'+str(i) for i in range(10)]
fig=plt.figure(figsize=(15, 10))
rows, cols = 3, 3
for i, image_path in enumerate(image_paths[:9]): # range(0,len(image_paths), len(image_paths)/16)
    track_id_and_bboxes = tracking_results[image_path]
    track_ids, bboxes = list(zip(*track_id_and_bboxes))
    image = cv2.imread(image_path)
    image = draw_detections(image, track_ids, track_names,  bboxes, COLORS_TRACKING)
    fig.add_subplot(rows, cols, i+1)
    plt.axis('off')
    plt.imshow(image)

##  <a id="sfmo"> 4) Retrouver les ellipsoides 3D</a>

C'est la dernière étape, et la plus importante, il s'agit à présent d'utiliser les détections en 2D de chaque image pour en tirer des ellipsoides en 3D.

Tout d'abord, il faut transformer chaque boite englobante en ellipse, encodée par une matrice symétrique 3x3 que nous vectorisons ensuite pour obtenir une vecteur 1x6

In [ ]:
import json
import numpy as np
from tp_geometry import sfmo
from tp_geometry import utils

# read the tracking results
tracking = json.load(open('tracking.json','r'))
n_o = 5 # I put 5 bottles on the table
n_f = len(tracking) # number of frames

# Construire une ellipse pour chaque boite englobantes et stocker les résultats dans une matrice C
C = np.zeros((n_f*3, n_o*3))
f = 0 # counting the number of images which are used
for (imgPath,dets) in tracking.items():
    if len(dets) != n_o:
        continue # I ignore images in which less than 5 objects have been detected.
    for (trackid, (x, y, w, h)) in dets:
        c = sfmo.bbx2ell((x, y, w, h))
        if np.abs(c.sum()) <= 0.1 or np.isnan(c).any():
            import pdb; pdb.set_trace()
        C[f*3:f*3+3, trackid*3:trackid*3+3] = c
    f += 1
C = C[:3*f] # f is the number of images which are actually used. We truncate the matrix to take only these elements.

# Vectoriser les differents composants de cette matrice. 
# C.a.d. que chaque matrice de conique 3x3 devient un vecteur 6x1
# La matrice passe donc de la dimension (n_f*3, n_o*3) à la dimension (nf_*6, n_o)
Cadjv_mat = utils.conics_to_vec(C, norm=True)


La prochaine étape est d'obtenir la position des centres en 3D et les matrices de caméras M à partir des centres en 2D des ellipses. Ouvrez le module `sfmo.py` et compléter le code de la fonction `sfm_from_centers` afin d'implémenter la méthode de Tomasi and Kanade.

Note : pour rechargez le module sans redémarrer le notebook, vous pouvez utiliser la librairie `importlib`
```
import importlib
importlib.reload(sfmo)
```

In [ ]:
# structure from motion for the centers of the conics
M, S = sfmo.sfm_from_centers(Cadjv_mat)

# Add additional orhogonality constraints, this is useful when the measurements are noisy
M, S = sfmo.fact_metric_constraint(M,S)

La matrice M contient les paramètres de la caméra pour projeter les centres. 

Nous pouvons l'utiliser pour construire une nouvelle matrice `G` qui liera les paramètres relatifs à la forme des ellipses à ceux des ellipsoides. 

L'extraction des paramètres liés à la forme des ellipses est effectuée par la fonction `center_ellipses` (il s'agit en fait de centrer les ellipses en fixant leur centres à 0). 

In [ ]:
# Build a rank 6 reduced matrix, eliminating rows and columns related to translation
Gred = sfmo.rebuild_Gred(M)

# Remove center from ellipses (it is equivalent to center ellipsoids in the orthographic case)
Ccenter = sfmo.center_ellipses(Cadjv_mat)

Il reste enfin à calculer la forme des ellipsoides grâce à la méthode de la pseudo inverse à partir du vecteur contenant la forme ellipses et la matrice `Gred`.


Comme nous utilisons les ellipses `Ccenter` qui sont centrées, les ellipsoides que nous obtenons sont aussi centrées. 
Nous ajoutons donc les centres 3D `S` obtenues précédemment avec la fonction `recombine_ellipsoids`

In [ ]:
# Get the shape
# votre code ici
Quadrics_centered = 

# add the centers
Rec  = sfmo.recombine_ellipsoids(Quadrics_centered,S)

### Affichage des résultats

Vous devriez à présent voir 5 ellipsoides allongées dont les positions respectent celle vues sur les images.

Encore une fois, Notez que le notebook n'est pas très adapté pour la visualisation 3D. Vous obtiendrez de meilleures visualisation en lançant le code depuis un terminal où matplotlib vous permet de faire pivoter la figure pour l'observer sous différents angles. 

In [ ]:
# convert the 4x4 matrices in structure which explicitly contain the centers, the axis lengths, and the orientation
ells = utils.quadrics2ellipsoids(Rec)
# Afficher le résultat
utils.plot_ellipsoids(ells)
